# Downloading Sentinel-2 data 

This notebook provides code on how to download Sentinel-2 data from Google Earth Engine (GEE). This specific example is closely connected to the creation of a fire propagation database. After creating the individual burned area propagation files, Sentinel-2 data is downloaded according to the shape and the geographical position of the burned area images. Nevertheless, the presented code can be used to download Sentinel-2 data for every other shape as well. 

The Sentinel-2 data is processed in GEE to create cloud-free composit images, before EEs computePixels function is used to transfer the data from server to client side. To increase the speed of this process, and to enable the download of Sentinel-2 data for large areas, the original shape is divided into a predefined number of equally sized squares. After the download, these squares are combined to reconstruct the original image.  

For fire propagation, information from other remote sensing collections is interesting, too. We therefore offer the option to include the Digital Elevation Model (DEM) from the TanDEM-X mission, as well as the Dynamic Land Cover classification to the downloaded Sentine-2 data. 

### 1. Importing the necessary libraries

While importing most necessary libraries can be done normally (e.g., using conda-forge and a specified conda environment), GDAL is used in geopandas, rasterio and rioxarray, which can sometimes lead to conflicts. This can be circumvented by following a specific order of installation. In the presented example, we used the Python Version 3.11.9 and installed the aformentioned packages in the following order:
1. conda install -c conda-forge gdal
2. conda install -c conda-forge geopandas rasterio
3. conda install -c conda-forge rioxarray

In [1]:
# import necessary libraries
import pandas as pd
import geopandas as geo

### 2. Import the modules for downloading Sentinel-2 data from Google Earth Engine

In [2]:
from sent2_from_ee import retrieve_sent2_from_ee

Read the geopandas dataframe that contains the shapes for which the Sentinel-2 data is supposed to be extracted.
This dataframe needs to be reprojected to the CRS 4326, since this is the CRS of Sentinel-2 in Google Earth Engine. 

If this code is directly used after creating the fire propagation database, gdf_poly already exists as a variable and it can be used accordingly. . 

In [9]:
gdf_poly = geo.read_file(".../shapes_of_burned_areas.shp")
gdf_poly_4326 = gdf_poly.to_crs(4326)

# The date is needed to extract the correct data from the GEE collection. This line ensures that the date column includes a datetime object
gdf_poly_4326["date"] = pd.to_datetime(gdf_poly_4326["date"])

To download Sentinel-2 from GEE, we wrote the 'retrieve_sent2_from_ee' class.
A new instance of this class needs to be initialized. 

The two necessary variables for initialization include:
1. A geopandas dataframe with the shapes for which you want to retrieve the Sentinel-2 data, and 
2. Your Google Earth Engine project name. This is needed for authentification

In [12]:
# initialize the retrieve_sent2_from_ee class
get_sent2 = retrieve_sent2_from_ee(burned_area_poly=gdf_poly_4326, proj_ee = 'your GEE project name')

Using the Google Earth Engine API with Python requires an authentification.
'authenticate_ee()' takes care of this. To work properly, ´the correct project name needs to be provided while initializing the new instance of the 'retrieve_sent2_from_ee class'.

In [13]:
# initialize to google earth engine
get_sent2.authenticate_ee()

After transferring the files from server to client (i.e., downloading the data from GEE), they are saved as raster files in the specified the output path. 

In this example, only specific Sentinel-2 bands are useful for fire related tasks. If other bands are needed, the list_of_bands parameter can be adjusted accordingly.

In [ ]:
# set path to save the raster files
out_path = ".../sentinel2"

# download the sentinel-2 data and save them in the output path.
# Here, the data is only downloaded for the first shape of the geodataframe.
# To download data for every shape of the geodataframe, the function 'download_sent2' needs to be iterate over every individual shape.
get_sent2.download_sent2(sel_polygon = gdf_poly_4326.iloc[0], 
                         start_date_fire = gdf_poly_4326.iloc[0].date, 
                         day_dif_for_composite_img = 29, 
                         num_sub_images = 2, 
                         out_path = out_path, 
                         list_of_bands = ["B2", "B3", "B4", "B8", "B11", "B12"],
                         crs = 4326)